In [1]:
import time
import pandas as pd
import dask.dataframe as dd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as po
import plotly.express as px
import os,sys
import json
from array import array

from collections import defaultdict
from scipy.signal import find_peaks
# Insert the path of modules folder 
# sys.path.insert(0, 'C:/Users/Jason/thesis_project')
sys.path.append("../")

from support import *
from features.feature_ts import genX
from experiment.algorithms.cluster_prep import *

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\os.py
c:\Users\Jason\thesis_project


In [4]:
X = genX([1994,2014], drop_0 = True)

Xbin = xBins(X, 'amd')

dropping all zero rows
[     987     1031     2009     7762     7796     8021  1002909  1002939
  1004190  1004891  1006448  1006511  1006512  1006513 12001515 12001801
 12001974 12005870]
[       2       13       39 ... 12028598 12028611 12028616]
[      18       20       23 ... 12028607 12029049 12029071]
[       1        6        7 ... 12028932 12028993 12029006]
[       3        4        5 ... 12029010 12029019 12029036]
[      15       16       17 ... 12029023 12029027 12029062]
[     179      180      182      186      187      200      202      203
      213      226      230      231      248      536      546      552
      561      567      579      581      690      693      695      708
      710      711      714      716      831      832     1235     3318
     4512     4513     4519     4524     4525     4529     4538     4539
     4546     4547     4562     4564     4566     4575     4576     4580
     4591     4595     4596     4597     4601     4607     4608     4610


In [121]:
X

0         1         2         3         4  \
ProfileID date                                                           
1         1994-06-18  3.691667  0.800000  0.508333  2.183333  0.741667   
          1994-06-19  0.808333  0.500000  2.075000  0.691667  0.325000   
          1994-06-20  1.058333  1.625000  0.341667  2.100000  1.225000   
          1994-06-21  0.425000  2.350000  0.550000  0.291667  2.100000   
          1994-06-22  0.741667  2.325000  0.583333  1.191667  1.225000   
...                        ...       ...       ...       ...       ...   
12029071  2014-09-29  0.400000  0.400000  0.193333  0.246667  1.733333   
          2014-09-30  0.400000  0.400000  0.400000  0.400000  0.400000   
          2014-10-01  0.380000  0.306667  0.280000  0.293333  0.273333   
          2014-10-02  0.293333  0.286667  0.286667  0.293333  1.433333   
          2014-10-03  0.353333  0.313333  0.253333  0.273333  0.386667   

                             5         6         7         8         9  ...  \
ProfileID date                                                          ...   
1         1994-06-18  0.658333  1.991667  0.700000  2.383333  5.033333  ...   
          1994-06-19  2.075000  0.691667  0.316667  3.075000  2.033333  ...   
          1994-06-20  1.058333  4.808333  4.091667  2.083333  0.475000  ...   
          1994-06-21  0.591667  4.066667  2.791667  0.816667  2.100000  ...   
          1994-06-22  0.575000  4.066667  3.350000  0.866667  2.808333  ...   
...                        ...       ...       ...       ...       ...  ...   
12029071  2014-09-29  2.886667  0.440000  0.340000  0.380000  0.313333  ...   
          2014-09-30  0.640000  0.620000  1.406667  1.566667  0.280000  ...   
          2014-10-01  1.166667  0.366667  2.533333  3.986667  0.626667  ...   
          2014-10-02  0.753333  0.200000  2.286667  0.340000  0.280000  ...   
          2014-10-03  0.213333  0.526667  0.500000  1.686667  0.473333  ...   

                            14        15        16        17        18  \
ProfileID date                                                           
1         1994-06-18  1.450000  1.133333  1.666667  3.916667  9.241667   
          1994-06-19  2.125000  2.616667  2.358333  1.141667  7.283333   
          1994-06-20  1.433333  2.641667  2.116667  5.541667  5.275000   
          1994-06-21  3.150000  1.825000  4.016667  1.833333  3.466667   
          1994-06-22  1.500000  0.933333  2.791667  6.291667  6.491667   
...                        ...       ...       ...       ...       ...   
12029071  2014-09-29  0.853333  0.773333  0.320000  0.406667  1.566667   
          2014-09-30  0.293333  0.233333  0.226667  0.360000  1.146667   
          2014-10-01  0.233333  0.280000  0.213333  0.280000  0.966667   
          2014-10-02  0.333333  0.246667  0.213333  0.340000  0.846667   
          2014-10-03  0.206667  0.220000  0.400000  0.380000  0.726667   

                            19         20         21        22        23  
ProfileID date                                                            
1         1994-06-18  3.275000   6.325000   2.816667  0.925000  2.025000  
          1994-06-19  5.391667   6.441667   9.716667  7.500000  0.841667  
          1994-06-20  6.533333   4.708333  10.433333  9.508333  4.408333  
          1994-06-21  8.875000  10.850000  12.908333  9.241667  2.625000  
          1994-06-22  4.458333  13.125000  10.750000  6.200000  1.350000  
...                        ...        ...        ...       ...       ...  
12029071  2014-09-29  0.686667   2.273333   1.620000  0.506667  0.400000  
          2014-09-30  0.980000   1.480000   1.886667  0.380000  0.293333  
          2014-10-01  1.726667   1.306667   3.000000  0.906667  0.340000  
          2014-10-02  2.200000   1.526667   0.646667  0.633333  0.340000  
          2014-10-03  0.660000   0.633333   0.686667  0.640000  0.360000  

[3252833 rows x 24 columns]

### Looking at users from a specific bin 600-1200

In [24]:
list_len = []
list_ids = []

for ids in Xbin['600-1200']:
    list_ids.append(ids)
    list_len.append(len(X.loc[ids,:]))

# Find the household that logged the most data
ix = list_len.index(max(list_len))

# Store the households ID
house_id = list_ids[ix]

# Create a dataframe with all of that households data
A = X.loc[house_id,:]

In [54]:
H = A.copy()
# H = A.reset_index()
# H.drop(['date'], axis = 1, inplace = True)

In [55]:
H

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
date,,,,,,,,,,,,,,,,,,,,,
1997-12-31,3.224250,3.260750,2.181083,3.260583,2.179667,4.138667,8.768333,11.164917,15.737083,21.388250,...,10.112500,15.238250,11.868167,24.005167,20.095917,16.590917,10.304833,10.285417,11.963250,12.727000
1998-01-01,9.764333,18.501500,6.569500,4.291500,3.257083,2.170750,3.766750,5.451583,26.613833,24.498083,...,7.819500,7.818083,8.930000,10.363333,9.704833,17.784667,18.773750,16.195167,4.450750,3.099500
1998-01-02,2.162417,3.143167,2.162417,3.245167,3.143417,4.620750,8.507750,16.953917,19.734000,30.708583,...,21.804333,23.030000,15.266583,24.452916,13.554250,13.445167,14.884583,10.891167,8.144083,3.418167
1998-01-03,2.156917,2.654750,3.230167,1.859917,3.236417,2.698917,8.202250,16.316167,13.867750,10.833167,...,8.840250,11.907833,6.852917,8.424083,23.650500,22.695167,15.322167,12.060250,12.837750,9.920583
1998-01-04,12.998500,13.494000,3.049583,2.065333,3.285250,1.829667,3.682000,6.419250,12.639917,16.389750,...,26.012833,25.863083,15.273667,14.706083,22.475000,25.642750,24.463500,14.018583,16.030833,10.670750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999-05-01,3.425167,2.136333,2.306167,1.561583,2.179833,1.550167,2.177333,1.527250,6.075333,10.894167,...,4.442750,3.316250,10.468667,14.200250,7.920667,14.141583,16.076750,14.424667,10.048750,13.720083
1999-05-02,1.102667,2.621417,2.176750,1.531583,2.151833,1.532083,2.371250,1.304500,5.697250,18.597750,...,12.435917,7.380750,7.464500,8.847500,12.890250,8.664750,12.021083,16.607000,14.113167,2.803500
1999-05-03,3.919667,1.639750,2.765500,1.270167,2.624500,1.345417,7.947750,7.871500,18.135667,20.612917,...,13.466500,14.680833,20.353000,29.992833,16.003750,17.046083,7.401750,12.110417,11.678417,15.560667


In [66]:
# temp = determinePeaks(H)

In [70]:
# temp.drop(['date_x','date_y', 'ToP_y'],axis = 1, inplace = True)

In [122]:
df = X.copy()
arrayX  = X.to_numpy().copy()
arrayY = X.to_numpy().copy()

listYM  = list(map(lambda x : x[:12],arrayY))
listYA  = list(map(lambda x : x[12:],arrayY))

arrayYM = np.array(listYM)
arrayYA = np.array(listYA)

# arrayYM.sort()
# arrayYA.sort()

dict_M = defaultdict(list)
dict_A = defaultdict(list)

df.reset_index(inplace = True)

for i in range(0,len(arrayX)):
    indices_M = np.where(arrayYM[i] == np.amax(arrayYM[i]))
    indices_A = np.where(arrayYA[i] == np.amax(arrayYA[i]))

    dict_M[i].append(indices_M[0][0]) 
    dict_A[i].append(indices_A[0][0])

ToP_M_df = pd.DataFrame.from_dict(dict_M)
ToP_A_df = pd.DataFrame.from_dict(dict_A)

top_M_dict = ToP_M_df.T
top_A_dict = ToP_A_df.T + 12

dict_M.clear() 
dict_A.clear()


X1 = df.merge(top_M_dict, left_index = True, right_index = True).merge(top_A_dict,left_index = True, right_index = True)

# X1.rename(columns = {'0_x':'ToP_M', '0_y':'ToP_A'}, inplace=True)

# X1['ToP'] = X1[['ToP_M', 'ToP_A']].values.tolist()
# ToP = X1.groupby(['date'])['ToP'].apply(pd.Series.tolist)
# temp = ToP.map(lambda x : [item for sublist in x for subsublist in sublist for item in subsublist])
# Temp1 = temp.to_frame()
# Temp1.reset_index(inplace = True)
# new_X1 = X1.merge(Temp1, right_index=True, left_index=True)
# new_X1.drop(['ToP_x'], axis = 1, inplace=True)
# new_X1['ToP_y'] = new_X1['ToP_y'].map(lambda x: [sublist.astype(str) for sublist in x])

In [123]:
X1

,ProfileID,date,0,1,2,3,4,5,6,7,...,16,17,18,19,20,21,22,23,0_x,0_y
0,1,1994-06-18,3.691667,0.800000,0.508333,2.183333,0.741667,0.658333,1.991667,0.700000,...,1.666667,3.916667,9.241667,3.275000,6.325000,2.816667,0.925000,2.025000,10,18
1,1,1994-06-19,0.808333,0.500000,2.075000,0.691667,0.325000,2.075000,0.691667,0.316667,...,2.358333,1.141667,7.283333,5.391667,6.441667,9.716667,7.500000,0.841667,10,21
2,1,1994-06-20,1.058333,1.625000,0.341667,2.100000,1.225000,1.058333,4.808333,4.091667,...,2.116667,5.541667,5.275000,6.533333,4.708333,10.433333,9.508333,4.408333,6,21
3,1,1994-06-21,0.425000,2.350000,0.550000,0.291667,2.100000,0.591667,4.066667,2.791667,...,4.016667,1.833333,3.466667,8.875000,10.850000,12.908333,9.241667,2.625000,6,21
4,1,1994-06-22,0.741667,2.325000,0.583333,1.191667,1.225000,0.575000,4.066667,3.350000,...,2.791667,6.291667,6.491667,4.458333,13.125000,10.750000,6.200000,1.350000,10,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252828,12029071,2014-09-29,0.400000,0.400000,0.193333,0.246667,1.733333,2.886667,0.440000,0.340000,...,0.320000,0.406667,1.566667,0.686667,2.273333,1.620000,0.506667,0.400000,5,20
3252829,12029071,2014-09-30,0.400000,0.400000,0.400000,0.400000,0.400000,0.640000,0.620000,1.406667,...,0.226667,0.360000,1.146667,0.980000,1.480000,1.886667,0.380000,0.293333,8,21
3252830,12029071,2014-10-01,0.380000,0.306667,0.280000,0.293333,0.273333,1.166667,0.366667,2.533333,...,0.213333,0.280000,0.966667,1.726667,1.306667,3.000000,0.906667,0.340000,8,21
3252831,12029071,2014-10-02,0.293333,0.286667,0.286667,0.293333,1.433333,0.753333,0.200000,2.286667,...,0.213333,0.340000,0.846667,2.200000,1.526667,0.646667,0.633333,0.340000,11,19


In [124]:
X1.iloc[3252831]

ProfileID               12029071
date         2014-10-02 00:00:00
0                       0.293333
1                       0.286667
2                       0.286667
3                       0.293333
4                       1.433333
5                       0.753333
6                            0.2
7                       2.286667
8                           0.34
9                           0.28
10                          0.24
11                      3.086667
12                      0.313333
13                      0.413333
14                      0.333333
15                      0.246667
16                      0.213333
17                          0.34
18                      0.846667
19                           2.2
20                      1.526667
21                      0.646667
22                      0.633333
23                          0.34
0_x                           11
0_y                           19
Name: 3252831, dtype: object

In [56]:
def determinePeaks(X):
    df = X.copy()
    arrayX  = X.to_numpy().copy()
    arrayY = X.to_numpy().copy()

    listYM  = list(map(lambda x : x[:12],arrayY))
    listYA  = list(map(lambda x : x[12:],arrayY))

    arrayYM = np.array(listYM)
    arrayYA = np.array(listYA)

    arrayYM.sort()
    arrayYA.sort()

    dict_M = defaultdict(list)
    dict_A = defaultdict(list)

    df.reset_index(inplace = True)

    for i in range(0,len(arrayX)):
        
        diff_M = arrayYM[i][len(arrayYM[i])-1] - arrayYM[i][len(arrayYM[i])-2]
        diff_A = arrayYA[i][len(arrayYA[i])-1] - arrayYA[i][len(arrayYA[i])-2]

        threshold_M = arrayYM[i][len(arrayYM[i])-1] - diff_M/3
        threshold_A = arrayYA[i][len(arrayYA[i])-1] - diff_A/3
        
        # distance = 2

        indices_M = find_peaks(arrayX[i][0:12], height = threshold_M)[0]
        indices_A = find_peaks(arrayX[i][12:], height = threshold_A)[0]

        dict_M[i].append(indices_M) 
        dict_A[i].append(indices_A)

    ToP_M_df = pd.DataFrame.from_dict(dict_M)
    ToP_A_df = pd.DataFrame.from_dict(dict_A)

    top_M_dict = ToP_M_df.T
    top_A_dict = ToP_A_df.T + 12

    dict_M.clear() 
    dict_A.clear()


    X1 = df.merge(top_M_dict, left_index = True, right_index = True).merge(top_A_dict,left_index = True, right_index = True)
    # X2 = X1.merge(top_A_dict,left_index = True, right_index = True)

    X1.rename(columns = {'0_x':'ToP_M', '0_y':'ToP_A'}, inplace=True)

    X1['ToP'] = X1[['ToP_M', 'ToP_A']].values.tolist()
    ToP = X1.groupby(['date'])['ToP'].apply(pd.Series.tolist)
    temp = ToP.map(lambda x : [item for sublist in x for subsublist in sublist for item in subsublist])
    Temp1 = temp.to_frame()
    Temp1.reset_index(inplace = True)
    new_X1 = X1.merge(Temp1, right_index=True, left_index=True)
    new_X1.drop(['ToP_x'], axis = 1, inplace=True)
    new_X1['ToP_y'] = new_X1['ToP_y'].map(lambda x: [sublist.astype(str) for sublist in x])

    return new_X1

In [57]:
# def plotPeaks(X, bin)
from plotly.subplots import make_subplots

def plotToPs(A, bin):
    
    temp = determinePeaks(A)

    ToPs = calculateToPs(temp)

    indices = temp['ToP_y']

    fig = make_subplots(rows=2, cols=1, subplot_titles=("Amplitude vs Time of Peak", "Histogram of ToPs"))

    # fig = go.Figure()

    def add_new_trace(index):
        fig.append_trace(go.Scatter(
        x=[int(i) for i in indices[index]],
        y=temp.loc[index,indices[index]].values,
        mode='markers',
        marker=dict(
            size=8,
            color='red',
            symbol='circle'
        ),
        name='Detected Peaks',
        showlegend = False
        ),row =1, col =1)  

    for i in temp.index:
        add_new_trace(i)

    fig.update_yaxes(title_text="Amplitude", row = 1, col = 1)

    fig.update_xaxes(title_text = "Time of Peak")

    fig.update_layout(title="Profiles for bin " + str(bin),
        xaxis = dict(
            tickmode = 'linear',
            tick0 = 0,
            dtick = 1
        )
    )

    #Plot Histogram of "total_bill" with rugplot parameters
    fig.append_trace(px.histogram(x=ToPs),row = 2, col =1)

    fig.update_yaxes(title_text="Count", row = 2, col = 1)

    # po.plot(fig, auto_open=False, filename=data_dir+'/plots/ToP_Amplitude_'+str(bin)+'.html')
    fig.show()

In [62]:
def calculateToPs(X1):
    
    X1['ToP'] = X1[['ToP_M', 'ToP_A']].values.tolist()
    ToP = X1.groupby(['date_x'])['ToP'].apply(pd.Series.tolist)
    temp = ToP.map(lambda x : [item for sublist in x for subsublist in sublist for item in subsublist])

    ToPs = temp.apply(pd.Series).stack().reset_index(drop = True)

    print(ToPs)

    return ToPs

In [63]:
plotToPs(H,'600-1200')
# print("2.Length of A is = {}".format(len(A)))

0       9.0
1      17.0
2      10.0
3      20.0
4       9.0
       ... 
874     9.0
875    17.0
876     8.0
877     9.0
878    16.0
Length: 879, dtype: float64


ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'hovertemplate': 'x=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'histogram',
              'x': array([ 9., 17., 10., ...,  8.,  9., 16.]),
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'count'}}}
})]

    The 'data' 